# Intro to Game AI and Reinforcement Learning

* Bu notebook kapsamında, kendi kendine oyun oynayan ajanlar oluşturacağız.
* Oyun ortamımızı oluşturacağız, ajanımızı oluşturacağız ve yapay zeka temelli oyunlar oluşturmak için geleneksel yöntemlere bakacağız. Öyle ki oluşturduğumuz ajanlar bir çok acemi oyuncuyu yenecek. :)
* Bölüm - 4 te ise Reinforcement Learning alanında en yeni algoritmaları deneyeceğiz.
* Hazırsanız başlayın !

In [ ]:
# Ortamımızı kurarken kaggle'dan yararlanacağız. İlk olarak anaconda prompt üzerinde pip install kaggle_environments diyerek
# paketi indirmek gerek. Sonrasında import edelim.

from kaggle_environments import make, evaluate

# Oyun ortamımızı create edelim.
env = make("connectx", debug=True)

# Kullanılabilir varsayılan araçlarımız.
print(list(env.agents))

In [ ]:
# İki rastgele ajan bir tur oyun oynuyor.
env.run(["random", "random"])
env.render(mode="ipython")

**Agent'ları Tanımlayalım.**

* Agent'ımızı iki bağımsız değişkeni kabul eden bir fonksiyon olarak tanımlayacağız."obs ve config"

In [ ]:
import random
import numpy as np

# baştaki sütunu seç.
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# orta sütunu seç.
def agent_middle(obs, config):
    return config.columns//2

# en son da ki sütunu seç.
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[0]

* **obs:** iki bilgi içerir. 
* *obs.board:* her ızgara konumu için yani 6*7 lik matrisin her elemanı için bir öge içeren bir python listesi.
* *obs.mark:* agent'a atanan parça 


* Yani işin özü şudur. oyun üzerinde anlık olarak bulunan ajanlardan mavi olanlara 1, sarı olanlara 2 değerini ata.

* **config** 3 adet bilgi içerir. 
* *config.columns: 7 kolon* , *config.rows: 6 satır*, *config.in a row: oyunu kazanmak için üst üste alınacak parça sayısı--> 4*

**Agent'ları Değerlendirelim.**

In [ ]:
# ajanlar bir oyun turu oynar.
env.run([agent_leftmost, agent_random])

# gösterelim.
env.render(mode="ipython")


Tek bir oyunun sonucu genellikle ajanlarımızın ne kadar iyi performans göstereceğini anlamak için yeterli bilgi değildir. Birden fazla oyunda her bir agent'ın kazanma yüzdelerini hesaplayacağız. Bunu yapmak için get_win_percentages() fonksiyonunu kullanacağız.

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # varsayılan connect four kurulumunu kullanıyoruz.
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 zaman ayarlaması.
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 zaman ayarlaması.      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Kazanma Yüzdesi:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Kazanma Yüzdesi:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Agent 1 için Geçersiz Oyun Sayısı:", outcomes.count([None, 0]))
    print("Agent 2 için Geçersiz Oyun Sayısı:", outcomes.count([0, None]))

In [ ]:
# rastgele oynayan agenta göre hangi agent'ımız daha fazla oyun kazanmış. bakalım!

get_win_percentages(agent1=agent_middle, agent2=agent_random)

In [ ]:
get_win_percentages(agent1=agent_leftmost, agent2=agent_random)

Şimdi ajanımıza oyun oynamayı öğretmeye başlayalım. Her olası geçerli hamleye bir puan atamak için sezgiselliği kullanacağız ve
en yüksek puanı alan hareketi seçeceğiz. (Birden fazla hamle yüksek puanı alırsa, rastgele birini seçeriz.)**One-step lookahead** ajanın sadece bir hamle uzağı hesapladığını ifade eder.


In [ ]:
import random
import numpy as np

# ajan seçilen sütuna bir parça düşürürse puan hesaplanır.
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = get_heuristic(next_grid, mark, config)
    return score

# Score_move için yardımcı işlev: ajan seçilen sütuna bir parça düşürürse bir sonraki adımda tahtayı alır.
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Score_move için yardımcı işlev: ızgara için sezgisel değerini hesaplar.
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
    return score

# Get_heuristic için yardımcı işlev: pencerenin sezgisel koşulları karşılayıp karşılamadığını kontrol eder
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Get_heuristic için yardımcı işlev: belirtilen sezgisel koşulları karşılayan pencere sayısını sayar
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [ ]:
# one-step lookahead tanımlanması

# Ajan her zaman iki bağımsız değişkeni kabul eden bir Python işlevi olarak uygulanır: obs ve config. 
# Bu işlemi yukarda yaptık zaten.
def agent(obs, config):
    # Geçerli hamlelerin listesini alın.
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Tahtayı 2D ızgaraya dönüştürün.
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Bir sonraki dönüşte olası her tahtaya bir puan atamak için sezgisel kullanın.
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Sezgisel değeri en üst düzeye çıkaran sütunların (hareketlerin) bir listesini alın.
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Sütunlardan rastgele seçelim.
    return random.choice(max_cols)

*Yukarıdaki kodu yorumlayalım**

* Agent geçerli hareketlerin bir listesini alarak başlar.
* Ardından oyun tahtasını 2D numpy dizisine dönüştürüyoruz.
* Sonra score_move() işlevi, geçerli her hareket için sezgisel değer hesaplar.
* drop_piece(), oynatıcı diskini seçilen sütuna bıraktığında oluşan ızgarayı döndürür.
* get_heuristic(), sağlanan kartın (grid) sezgisel değerini hesaplar; burada işaret, agent'ın işaretidir. Bu işlev, buluşsaldan belirli koşulları karşılayan pencere sayısını (satır, sütun veya köşegendeki dört bitişik konumdan) sayan count_windows() işlevini kullanır. Özellikle, count_windows (grid, num_discs, piece, config), oyun tahtasındaki (grid), mark piece ile oynatıcıdan (agent veya rakip) num_discs parçaları içeren ve pencerede kalan konumların boş olduğu pencere sayısını verir. Örneğin,
* num_dıscs =4 ve piece = obs ayarı.mark, ajanın üst üste dört disk alma sayısını sayar.
* num_dıscs = 3 ve piece = obs ayarı.%2+1 işareti, rakibin üç diski olduğu ve kalan konumun boş olduğu pencere sayısını sayar (rakip boş noktayı doldurarak kazanır).
* Son olarak, sezgisel olanı en üst düzeye çıkaran ve rastgele bir tane (tekdüze) seçen sütunların listesini alırız.

* (Not: Yukarıdaki kodu anlamak için zaman ayırdıktan sonra, çok daha hızlı çalışmasını sağlamak için nasıl yeniden yazacağınızı görebiliyor musunuz? İpucu olarak, count_windows() işlevinin oyun tahtasındaki konumların üzerinde döngü yapmak için birkaç kez kullanıldığını unutmayın.)

In [ ]:
from kaggle_environments import make, evaluate

env = make("connectx")

env.run([agent, "random"])

env.render(mode="ipython")

In [ ]:
# performanslara göz atalım.

def get_win_percentages(agent1, agent2, n_rounds=100):
   
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
           
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
        
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Kazanma:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Kazanma:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Agent 1 Geçersiz:", outcomes.count([None, 0]))
    print("Agent 2 Geçersiz:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=agent, agent2="random")

Şimdi ise agent'ımızın başarısını önemli ölçüde artıracak olan oyun teorisindeki min-max işlevini kullanacağız. **Minimax** şu şekilde çalışır.Oyun ağacının derinliklerinden gelen bilgileri kullanmak istiyoruz. Şimdilik 3 derinlikte çalıştığımızı varsayalım. Bu şekilde, ajan hareketine karar verirken, ajan tüm olası oyun tahtalarını dikkate alır. Bu durumda ajan kendi hamlesini, rakibin olası hamlelerini ve bir sonraki kendi hamlesini hesaplar.

In [ ]:
# önceki kodun birebir aynısını aldık.

import random
import numpy as np

def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid


def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    

def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

* Ayrıca, önceki bölümden sezgiselliği değiştirmemiz gerekecek, çünkü rakip artık oyun tahtasını değiştirebiliyor.

In [ ]:
# Minimax için yardımcı işlev: oyun alanı için sezgisel değer hesaplar.
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

In [ ]:
# Minimax için ihtiyaç duyacağımız birkaç ek işlev.

# Uses minimax to calculate value of dropping piece in selected column
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

# Minimax implementation
def minimax(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return get_heuristic(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax(child, depth-1, False, mark, config))
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            value = min(value, minimax(child, depth-1, True, mark, config))
        return value

In [ ]:

N_STEPS = 3 # agent'a 3 hamle düşün dedik. Ne kadar derin , o kadar süre :d

# agent tanımlamasını bir önceki bölümlerde bir kaç kez yaptık.
def agent(obs, config):
    
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    return random.choice(max_cols)

In [ ]:
# oyun zamanı .

from kaggle_environments import make, evaluate

env = make("connectx")

env.run([agent, "random"])

env.render(mode="ipython")

In [ ]:
# performans hesabı.

def get_win_percentages(agent1, agent2, n_rounds=100):
    
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
             
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
          
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Kazanma:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Kazanma:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Agent 1 Kaybetme:", outcomes.count([None, 0]))
    print("Agent 2 Kaybetme:", outcomes.count([0, None]))

In [ ]:
# 50 oyun için hesaplayalım.

get_win_percentages(agent1=agent, agent2="random", n_rounds=50)

Şimdi ise **Deep Reinforcement Learning** konusuna bakacağız.Sezgisellik kullanmadan akıllı bir ajan oluşturmak için takviye öğrenimini nasıl kullanacağınızı öğreneceğiz. Sadece oyunu oynayarak ve kazanma oranını en üst düzeye çıkarmaya çalışarak, zaman içinde ajanın stratejisini kademeli olarak geliştireceğiz.


* Derin sinir ağımız (DNN) geçerli oyun tahtasındaki dizilimi girdi kabul eder ve olası her hareket için bir olasılık verir.
* Agent bu olasıklıklardan bir seçim yapar. Yüksek olasılığın seçimi daha yüksektir.
* Bu şekilde, iyi bir oyun stratejisini kodlamak için, sadece ağın ağırlıklarını değiştirmemiz gerekir, böylece mümkün olan her oyun tahtası için daha iyi hamlelere daha yüksek olasılıklar atar.( derin öğrenme )

**Peki soru şu? Ağın ağırlıklarını nasıl güncelleriz ?**

* Her hareketten sonra, ajana ne kadar iyi yaptığını söyleyen bir ödül veriyoruz.
* Eğer ajan bu hamlede oyunu kazanırsa, ona +1'lik bir ödül veririz.
* Aksi takdirde, ajan geçersiz bir hamle yaparsa (oyunu bitirir), ona -10'luk bir ödül veririz.
* Rakip bir sonraki hamlesinde oyunu kazanırsa (yani, ajan rakibinin kazanmasını engelleyemedi), ajana -1'lik bir ödül veririz.
* Aksi takdirde, ajan 1/42'lik bir ödül alır.(oyun tahtası 6*7 lik.)
* Her oyunun sonunda, ajan ödülünü ekler. Ödüllerin toplamını ajanın kümülatif ödülü olarak adlandırıyoruz.
* **Örneğin:** 
* Oyun 8 hamle sürdüyse (her oyuncu dört kez oynadı) ve ajan nihayetinde kazanırsa, kümülatif ödülü 3 * (1/42) + 1'dir. 
* Oyun 11 hamle sürdüyse (ve rakip ilk önce oynadı, böylece ajan beş kez oynadı) ve rakip son hamlesinde kazandıysa, ajanın kümülatif ödülü 4 * (1/42) - 1'dir.
* Oyun berabere biterse, ajan tam olarak 21 hamle oynadı ve 21 * (1/42) kümülatif ödül aldı.
* Oyun 7 hamle sürdüyse ve ajan geçersiz bir hamle seçtiğinde sona ererse, ajan 3 * (1/42) - 10 kümülatif ödül alır.
* **AMACIMIZ:** Ortalama olarak ajanın kümülatif ödülünü en üst düzeye çıkaran sinir ağının ağırlıklarını bulmaktır.

Bir ajanın performansını izlemek için ödül kullanma fikri, **takviye öğrenimi - reinforcement learning** alanında temel bir fikirdir. Sorunu bu şekilde tanımladıktan sonra, bir aracı üretmek için çeşitli takviye öğrenme algoritmalarından herhangi birini kullanabiliriz.

**Reinforcement Learning / Takviyeli Öğrenme**

**DQN**, **A2C** ve **PPO** gibi birçok farklı **takviye öğrenme algoritması** vardır. Tüm bu algoritmalar bir agent üretmek için benzer bir işlem kullanır.

* Başlangıçta, ağırlıklar rasgele değerlere ayarlanır.
* Agent oyunu oynarken algoritma, kümülatif ödülün ortalama olarak nasıl etkilendiğini görmek için ağırlıklar için sürekli olarak yeni değerler dener. Zamanla, birçok oyun oynadıktan sonra, ağırlıkların kümülatif ödülü nasıl etkilediğine dair iyi bir fikir edinir ve algoritma daha iyi performans gösteren ağırlıklara doğru yerleşir.
* Bu şekilde, oyunu kazanmaya çalışan bir ajan ile ortaya çıkar. (böylece +1'in son ödülünü alır ve -1 ve -10'dan kaçınır) ve oyunu mümkün olduğunca uzun süre sürdürmeye çalışır (böylece 1/42 bonusunu toplayabilir).

**Uygulama**

PPO algoritmasını kullanacağız. Tensorflow 1 ile devam edeceğiz. Uygualama tensorflow 2 yi desteklememekte.

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install tensorflow==1.15.0
!pip install gym==0.21.0

In [ ]:
# Check version of tensorflow
import tensorflow as tf
tf.__version__

In [ ]:
from kaggle_environments import make, evaluate
from gym import spaces

class ConnectFourGym:
    def __init__(self, agent2="random"):
        ks_env = make("connectx", debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(self.rows,self.columns,1), dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    def reset(self):
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(self.rows,self.columns,1)
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        return np.array(self.obs['board']).reshape(self.rows,self.columns,1), reward, done, _

 **Yukarıdaki kod ne yapar?**

ConnectFourGym sınıfı connectx'i OpenAI GYM ortamı olarak uygular ve çeşitli yöntemler kullanır:
* reset() : her oyunun başında çağrılır. başlangıç oyun tahtasını 6 satır ve 7 sütun içeren bir 2D numpy dizisi olarak döndürür.
* change_reward() : agentın aldığı ödülleri özelleştirir. (Yarışmanın, agentları sıralamak için kullanılan ödüller için zaten kendi sistemi vardır ve bu yöntem, değerleri tasarladığımız ödül sistemine uyacak şekilde değiştirir.)
* step(), rakibin tepkisi ile birlikte ajanın eylem seçimi için kullanılır.

Şimdi rastgele ajanı yenmek için bir ajan yetiştireceğiz. Bu rakibi aşağıdaki agent2 argümanında belirtiyoruz.

In [ ]:
env = ConnectFourGym(agent2="random")

* Kararlı Taban Çizgileri oyunu, "vektörize" ortamlarla çalışmamızı gerektirir. Bunun için Dummyvevent sınıfını kullanabiliriz.
* Monitor sınıfı, gittikçe daha fazla oyun oynarken aracı'nın performansının nasıl kademeli olarak geliştiğini izlememize olanak tanır.

In [ ]:
!apt-get update
!apt-get install -y cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install "stable-baselines[mpi]==2.10.2"

In [ ]:
import os
from stable_baselines.bench import Monitor 
from stable_baselines.common.vec_env import DummyVecEnv

# Eğitim bilgilerini günlüğe kaydetmek için dizin oluştur.
log_dir = "ppo/"
os.makedirs(log_dir, exist_ok=True)

# İlerlemeyi günlüğe kaydet
monitor_env = Monitor(env, log_dir, allow_early_resets=True)

# Vektörize bir ortam oluşturun
vec_env = DummyVecEnv([lambda: monitor_env])

Bir sonraki adım, sinir ağının mimarisini belirlemektir. Bu durumda, konvolüsyonel bir sinir ağı kullanıyoruz.Bunun, her bir sütunu seçme olasılıklarını veren sinir ağı olduğunu unutmayın. PPO algoritmasını kullandığımızdan (aşağıdaki kod hücresinde PPA1), ağımız bazı ek bilgiler de (girişin "değeri" olarak adlandırılır) çıkaracaktır.

In [ ]:
from stable_baselines import PPO1 
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc
from stable_baselines.common.policies import CnnPolicy

# Eylem değerlerini tahmin etmek için sinir ağı ( CNN )
def modified_cnn(scaled_images, **kwargs):
    activ = tf.nn.relu
    layer_1 = activ(conv(scaled_images, 'c1', n_filters=32, filter_size=3, stride=1, 
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = activ(conv(layer_1, 'c2', n_filters=64, filter_size=3, stride=1, 
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = conv_to_fc(layer_2)
    return activ(linear(layer_2, 'fc1', n_hidden=512, init_scale=np.sqrt(2)))  

class CustomCnnPolicy(CnnPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomCnnPolicy, self).__init__(*args, **kwargs, cnn_extractor=modified_cnn)
        
# Agent'ı başlat.
model = PPO1(CustomCnnPolicy, vec_env, verbose=0)

* Yukarıdaki kod hücresinde, sinir ağının ağırlıkları başlangıçta rasgele değerlere ayarlanır.
* Ajanın eğitim sırasında aldığı kümülatif ödülün yuvarlanma ortalamasını çiziyoruz. Artan fonksiyonun kanıtladığı gibi, ajan yavaş yavaş oyunu oynayarak daha iyi performans göstermeyi öğrendi.

In [ ]:
# ajan'ın eğitimi.
model.learn(total_timesteps=100000)

# toplam ödülü çizelim.
with open(os.path.join(log_dir, "monitor.csv"), 'rt') as fh:    
    firstline = fh.readline()
    assert firstline[0] == '#'
    df = pd.read_csv(fh, index_col=None)['r']
df.rolling(window=1000).mean().plot()
plt.show()

In [ ]:
# Son olarak, eğitimli ajanı yarışma için gereken formatta belirtiyoruz.

def agent1(obs, config):
    # Bir sütun seçmek için en iyi modeli kullanın.
    col, _ = model.predict(np.array(obs['board']).reshape(6,7,1))
    # Seçili sütunun geçerli olup olmadığını kontrol edin.
    is_valid = (obs['board'][int(col)] == 0)
    # Geçerli değilse rastgele seçin.
    if is_valid:
        return int(col)
    else:
        return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])

In [ ]:
# oyun zamanı..

env = make("connectx")

env.run([agent1, "random"])

env.render(mode="ipython")

In [ ]:
# ve performans hesabı..

def get_win_percentages(agent1, agent2, n_rounds=100):
    
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
             
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
          
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Kazanma:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Kazanma:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Agent 1 Kaybetme:", outcomes.count([None, 0]))
    print("Agent 2 Kaybetme:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=agent1, agent2="random")

Daha fazlası için : https://openai.com/blog/competitive-self-play/